In [1]:
from lark import Lark
from lark.visitors import Transformer, v_args
from __future__ import annotations

In [6]:
class FieldAndTale:
    def __init__(self, field:str, table: str, group_by: str, pos: int, tables=None):
        self.field = field
        self.value_table = table
        self.tables = set([table])
        if tables is not None:
            self.tables = tables
        self.group_by = group_by
        self.pos = pos
        self.withs = [] # 縦横変換用 サブクエリテーブル定義

    def __str__(self):
        return ", ".join(self.withs) + f"SELECT {self.field} FROM {', '.join(self.tables)} GROUP BY {self.value_table}.{self.group_by};"
    
    def add_unary(self, unary: str) -> FieldAndTale:
        self.field = f"{unary}({self.field})"
        return self

    @staticmethod
    def merge_FieldAndTable(a: FieldAndTale, ope: str, b: FieldAndTale, pos: int):
        field = f"({a.field}){ope}({b.field})"
        tables = a.tables | b.tables
        return FieldAndTale(field, a.value_table, a.group_by, pos, tables)

In [7]:
# インタプリタの本体
# SQL文に変換する
class ExprTransformer(Transformer):
    def __init__(self, table_name:str, group_by: str, visit_tokens = True):
        super().__init__(visit_tokens)
        self.table_name = table_name
        self.group_by = group_by
    @v_args(meta=True)
    def max(self, meta, args):
        # TODO: ★項目間関数群をUNIONを使って縦積みして計算するように修正
        return f"MAX({', '.join(args)})"
    @v_args(meta=True)
    def min(self, meta, args):
        return f"MIN({', '.join(args)})"
    @v_args(meta=True)
    def mean(self, meta, args):
        return f"MEAN({', '.join(args)})"
    @v_args(meta=True)
    def median(self, meta, args):
        return f"MEDIAN({', '.join(args)})"
    @v_args(meta=True)
    def add(self, meta, args):
        # return f"({args[0]}) + ({args[1]})"
        return FieldAndTale.merge_FieldAndTable(args[0], "+", args[1], meta.column)
    @v_args(meta=True)
    def sub(self, meta, args):
        # return f"({args[0]}) - ({args[1]})"
        return FieldAndTale.merge_FieldAndTable(args[0], "-", args[1], meta.column)
    @v_args(meta=True)
    def mul(self, meta, args):
        # return f"({args[0]}) * ({args[1]})"
        return FieldAndTale.merge_FieldAndTable(args[0], "*", args[1], meta.column)
    @v_args(meta=True)
    def div(self, meta, args):
        # return f"({args[0]}) / ({args[1]})"
        return FieldAndTale.merge_FieldAndTable(args[0], "/", args[1], meta.column)
    @v_args(meta=True)
    def unary_minus(self, meta, args):
        # return f"-({args[0]})"
        return args[0].add_unary("-")
    @v_args(meta=True)
    def unary_plus(self, meta, args):
        # return f"+({args[0]})"
        return args[0].add_unary("+")
    @v_args(meta=True)
    def number(self, meta, args):
        # return str(float(args[0]))
        return FieldAndTale(field=args[0], table=self.table_name, group_by=self.group_by, pos=meta.column)
    @v_args(meta=True)
    def symbol(self, meta, args):
        # return str(args[0])
        # 計測項目については、データが入っているテーブル名を指定
        return FieldAndTale(field=f"{args[0]}.{self.table_name}", table=self.table_name, group_by=self.group_by, pos=meta.column)

In [8]:
args = ["", "sql.txt"]
file_name = args[1]
with open("./testlark.lark", encoding="utf-8") as grammar:
    with open("./"+file_name,encoding="utf-8") as file:
        # text=file.read().replace("\n","").replace(" ","").replace("\t","") # 改行、スペース、タブは排除
        text=file.read()
        # parser = Lark(grammar.read(), parser='lalr', start="expr", transformer=ExprTransformer()) # 式のみ
        # result = parser.parse(text)
        parser = Lark(grammar.read(), parser='lalr', start="expr", propagate_positions=True)
        tree = parser.parse(text)
        # データが入っているテーブル名と集計単位用フィールドを指定して変換
        result = ExprTransformer("hive_table", "wafer_id").transform(tree)
        print(result)

SELECT (-(1.5))+(TEST_1.hive_table) FROM hive_table GROUP BY hive_table.wafer_id;
